# Testbench

# Requirements

## Packages

    sudo apt install python3-serial
    sudo apt install gcc-arm-none-eabi binutils-arm-none-eabi
    pip install progress

## Permissions

Add yourself to the `dialout` group:

    sudo usermod -a -G dialout $USER

# (Slave) Interface

## Slave interface

The target device is foremost a slave of the testbench. 
The **slave interface** is the interface of target where it receives it's APDUs.
The slave interface can be chosen by a **jumper conguration**.
The jumper configuration is evaluated in the `main()` function in order to call `slave_init()` with the suitable `protocol` argument.

The following are target invariant:

    - apdu.c, apdu.h (apdu_receive(), apdu_loop())
    - `slave_interface.h`
    - `slave_interface` methods except `slave_init()`.
    

# Sam3x8e (Arduino Due)

## Building

Run in the `testbench` folder:

    make bootloader
    make sam3x8e-libflash
    make sam3x8e-flash.elf

## Programming

The python package contains flashing without an external programmer. Run in the `testbench` folder:

    make sam3x8e-flash PORT=ttyACM0

## Testing

Sanity tests with the Loader client: 

In [4]:
from py import *
l = loader.loader("/dev/ttyACM0")

In [5]:
l.echo(b"abc")

b'abc'

In [7]:
hex(l.read_word(0x2000_0000))

'0x29e50d2e'

## C Standard library

Findet sich für `arm-none-eabi` unter `/usr/lib/gcc/arm-none-eabi/12.2.1/include`

In [1]:
ls /usr/lib/arm-none-eabi/include/

alloca.h   elf.h       ieeefp.h    newlib.h           signal.h     tar.h
_ansi.h    envlock.h   inttypes.h  _newlib_version.h  spawn.h      termios.h
argz.h     envz.h      langinfo.h  paths.h            ssp/         tgmath.h
ar.h       errno.h     libgen.h    pthread.h          stdatomic.h  threads.h
assert.h   fastmath.h  limits.h    pwd.h              stdint.h     time.h
bits/      fcntl.h     locale.h    reent.h            stdio_ext.h  unctrl.h
c++/       fenv.h      machine/    regdef.h           stdio.h      unistd.h
complex.h  fnmatch.h   malloc.h    regex.h            stdlib.h     utime.h
cpio.h     getopt.h    math.h      rpc/               string.h     utmp.h
ctype.h    glob.h      memory.h    sched.h            strings.h    wchar.h
devctl.h   grp.h       nano/       search.h           sys/         wctype.h
dirent.h   iconv.h     ndbm.h      setjmp.h           _syslist.h   wordexp.h


# HT32F52

## Hardware

We use the [HT32 Starter Kit](https://www.holtek.com.tw/WebAPI/bf86d133-ffb5-41e0-9ec2-0a9a29f075a7?_gl=1*x1jh1m*_ga*NTQwODExMTU3LjE3Mzg1OTIxMjg.*_ga_QJGHM3VFKP*MTczODU5MjEyNy4xLjAuMTczODU5MjEyNy4wLjAuMA..*_ga_2HPXRBSN2N*MTczODU5MjEyNy4xLjAuMTczODU5MjEyNy4wLjAuMA..).

Instead of the e-Link32 lite interface on the ESK32-30501 eval board, we use a seperate [CP210x USB to UART bridge] with the follwing connections:

 CP210x | ESK32-30501 | HT32F52352
 ------ | ----------- | ----------
 3V3    | VDD         | ...
 DTR    | nRST        | ...
 RXI    | ...         | ...
 TXD    | ...         | ...
 GND    | GND         | ...

## Juper Configuration

- To start the ISP bootloader at power up, connect pin A9 to GND.
- To select SPI as slave interface connect pin C1 to VDD.

## Python Classes

Class        | Python Module          | Brief description
------------ | ---------------------- | ------------------------------------------
ISP          | py/ht32/isp.py         | Bootloader client for use in flash.py
Ht32Serial   | py/ht32/ht32serial.py  | Wraps around serial.Serial
Apdu         | py/apdu.py             | APDU client
Target       | py/target.py           | Standard commands or prototypes
H32Target    | py/ht32/ht32target.py  | HT32 specific commands or implementations

## Building and programming

In the `testbench`folder call:

    make ht32-flash.bin
    make ht32-flash PORT=ttyUSB0

## Testing

In [8]:
import py

In [9]:
s = py.Ht32Serial("/dev/ttyUSB0")

In [10]:
h = loader.Loader(s)

In [11]:
h.echo(b"abcdefg")

b'abcdefg'

# Using I2C as slave interface

In [9]:
%%bash
git log --oneline --decorate -n 1

0be3853 (HEAD -> interface) sam i2c communication baeteen master and slave is working


In [10]:
%%bash
shasum sam3x8e/ram.bin

ebaed0d46ed90aece87aa2e1bc5e3253ab83b740  sam3x8e/ram.bin


In [4]:
import testbench
import testbench.adapter

In [3]:
s0 = testbench.rpcserial.Serial('/dev/ttyACM0') # the debugger, connected to master 

In [4]:
s1 = testbench.rpcserial.Serial('/dev/ttyACM1') # the slave
s2 = testbench.rpcserial.Serial('/dev/ttyACM2') # the master

In [11]:
rl = testbench.sam3x8e.ramloader.RamLoader(s2)

In [12]:
rl.load(0x20071000, 'sam3x8e/ram.bin', go=True) # takes a few seconds

In [13]:
rl = testbench.sam3x8e.ramloader.RamLoader(s1)

In [14]:
rl.load(0x20071000, 'sam3x8e/ram.bin', go=True) # takes a few seconds

In [15]:
t1 = testbench.target.Target(s1)

In [16]:
t1.i2c_open()

In [17]:
t1.i2c_send(b'ABCDEFGHIJ') # will not return within 2 seconds

Timeout: Received 0 bytes in 2 seconds but expected 2.

In [20]:
a = testbench.adapter.Adapter(s2)

In [21]:
a.open()

In [22]:
a.read(10)

b'ABCDEFGHIJ'

In [23]:
s1.read_all() # late response to `t1.i2c_send()`

b'\x00\x90'

In [24]:
t1.i2c_receive(4)

Timeout: Received 0 bytes in 2 seconds but expected 6.

In [25]:
a.write(b'abcd')

In [26]:
s1.read_all()

b'abcd\x00\x90'

In [27]:
s0.close()
s1.close()
s2.close()

# APDU via I2C
Bridge Due Pins A5 and A6 to selct I2C as slave interface.

In [1]:
%%bash
git log --oneline --decorate -n 1

ace1e26 (HEAD -> interface, origin/interface) handle apdus on I2C now working on SAM3X8E with slave address 0x11


In [2]:
%%bash
shasum sam3x8e/flash.bin

ac5f5ef7b26175b06b96ba56efba8e8f4fd8b982  sam3x8e/flash.bin


In [5]:
s2 = testbench.rpcserial.Serial('/dev/ttyACM2') # the master

In [6]:
a = testbench.adapter.Adapter(s2)

In [7]:
a.open()

In [8]:
t = testbench.target.Target(a)

In [9]:
t.echo(b'abcdefg')

b'abcdefg'

# Tinycryptlib

## EC Data structures

tc_sha256_state_struct

Functions to change between data types

    uECC_vli_bytesToNative
    uECC_vli_nativeToBytes

Data types

    uECC_word_t             unsigned int
    struct uECC_Curve_t;    see lib/include/tinycrypt/ecc.h

```
struct uECC_Curve_t {
    wordcount_t num_words;
    wordcount_t num_bytes;
    bitcount_t num_n_bits;
    uECC_word_t p[NUM_ECC_WORDS];
    uECC_word_t n[NUM_ECC_WORDS];
    uECC_word_t G[NUM_ECC_WORDS * 2];
    uECC_word_t b[NUM_ECC_WORDS];
    void (*double_jacobian)(uECC_word_t * X1, uECC_word_t * Y1, uECC_word_t * Z1,
        uECC_Curve curve);
    void (*x_side)(uECC_word_t *result, uECC_word_t *x, uECC_Curve curve);
    void (*mmod_fast)(uECC_word_t *result, uECC_word_t *product);
};
```


- hex strings im C-Code
- string2scalar: hex string to unsigned int vector
    - strlen(str)
    - hex2bin: hex string to bytes vector
    - uECC_vli_bytesToNative



## RNG

Wird nur benutzt in:

 - Schlüsselerzeugung in Funktion `uECC_make_key (ecc_dh.c)`
 - Erzeugung von Zufallszahl `k` in `uECC_sign (ecc_dsa.c)`

Für SCA-Tests kann beides in der Testbench erzeugt werden, dann ist nur 
`uECC_sign_with_k` zu testen.

# Adapter

Use the Arduino Due as USB to USART `adapter` with a second Arduino Due as `target` device. 

- The master interface of `adapter` is USART1, i.e. Pins 16 (TX2) and 17 (RX2).
- The slave interface of `target` device is USART1, i.e. Pins 17 (RX2) and 16 (TX2).
- Cross-Connect the Pins 16 and 17 of the two devices.
- Connect GND of the two devices.
- Connect Due Pin A6 (analog side) of `target`to GND to select USART1 as slave interface.

